In [ ]:
import numpy as np
import sncosmo
from astropy.table import Table
from matplotlib import pyplot as plt

import sys; sys.path.insert(0, '../')
from data_access import sdss


## Get SDSS published table of spectroscopically confirmed SNIa

In [ ]:
spec_confirmed_sn = sdss.master_table[sdss.master_table['Classification'] == 'zSNIa']
spec_confirmed_sn.show_in_notebook(display_length=10)


## Pick an arbitrary target and look at the light curve

In [ ]:
test_id = 932

# Get fit published values
published_values = sdss.master_table[sdss.master_table['CID'] == test_id]
x0 = published_values['x0SALT2zspec'][0]
x1 = published_values['x1SALT2zspec'][0]
c = published_values['cSALT2zspec'][0]
chisq_norm = published_values['chi2SALT2zspec'][0] / published_values['ndofSALT2zspec'][0]
peak_mjd = published_values['MJDatPeakrmag'][0]

print('Published Values for CID {}:'.format(test_id))
print('x0: ', x0)
print('x1: ', x1)
print('c: ', c)
print('chisq: ', published_values['chi2SALT2zspec'][0])
print('ndof: ', published_values['ndofSALT2zspec'][0])
print('chisq_norm: ', chisq_norm)

# Get sdss photometry data and apply cuts described in Sako et al,
phot_data = sdss.get_data_for_id(test_id)
phot_data = phot_data[phot_data['FLAG'] < 1024]
#phot_data = phot_data[phot_data['MJD'] < peak_mjd + 45]
#phot_data = phot_data[phot_data['MJD'] > peak_mjd - 15]

fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    band_data = phot_data[phot_data['FILT'] == i]
    axes[i].scatter(band_data['MJD'], band_data['MAG'])
    axes[i].errorbar(band_data['MJD'], band_data['MAG'], yerr=band_data['MERR'], linestyle='')

plt.show()

phot_data.show_in_notebook(display_length=10)


## Create an SNCosmo input table

In [ ]:
@np.vectorize
def sdss_mag_to_ab_flux(mag, band):
    """For a given sdss magnitude return the AB flux

    Args:
        mag (float): An SDSS asinh magnitude
        band  (str): The band of the magnitude doi_2010_<ugriz><123456>
        
    Return:
        The equivalent AB magnitude
    """

    if band[-2] == 'u':
        offset = -0.679

    elif band[-2] == 'g':
        offset = 0.0203

    elif band[-2] == 'r':
        offset = 0.0049

    elif band[-2] == 'i':
        offset = 0.0178

    elif band[-2] == 'z':
        offset = 0.0102

    else:
        ValueError('Unknown band {}'.format(band))

    return 3631 * 10 ** ((mag + offset) / -2.5)


def calc_err(sigma_sdss_mag, flux_ab):
    """Calculate the error of the AB magnitude equivilent for an SDSS asinh mag 
    
    Args:
        sigma_sdss_mag (float): Error in the SDSS magnitude
        flux_ab        (float): AB flux of the measurement
        
    Returns:
        The error in the equivalent AB flux
    """

    return sigma_sdss_mag * flux_ab * np.log(10) / 2.5


@np.vectorize
def band_name(filt, idccd):
    """Return the sncosmo band name given filter and CCD id
    
    Args:
        filt  (str): Filter name <ugriz>
        idccd (int): Column number 1 through 6
    
    Args:
        The name of the filter registered with sncosmo
    """

    return 'doi_2010_{}{}'.format('ugriz'[filt], idccd)

In [ ]:
input_table = Table()
input_table.meta = phot_data.meta
input_table['time'] = phot_data['MJD']
input_table['band'] = band_name(phot_data['FILT'], phot_data['IDCCD'])
input_table['zp'] = np.full(len(phot_data), 2.5 * np.log10(3631))
input_table['flux'] = sdss_mag_to_ab_flux(phot_data['MAG'], input_table['band'])
input_table['fluxerr'] = calc_err(phot_data['MERR'], input_table['flux'])
input_table['zpsys'] = np.full(len(phot_data), 'ab')

input_table.show_in_notebook(display_length=10)


## Run fit

In [ ]:
print('\n\nFitting for all terms except z:')
source = sncosmo.get_source('salt2', version='2.0')
model = sncosmo.Model(source=source)
model.set(z=input_table.meta['redshift'])

result, fitted_model = sncosmo.fit_lc(input_table, model, ['t0', 'x0', 'x1', 'c'], bounds=None)
result['chi2_norm'] = (result.chisq / result.ndof)
sncosmo.plot_lc(input_table, model=fitted_model, errors=result.errors)
plt.show()
print(result)

print('\n\nFitting with fixed, published values:')
model.set(z=input_table.meta['redshift'], x0=x0, x1=x1, c=c)
result, fitted_model = sncosmo.fit_lc(input_table, model, ['t0'], bounds=None)
result['chi2_norm'] = (result.chisq / result.ndof)
sncosmo.plot_lc(input_table, model=fitted_model, errors=result.errors)
plt.show()
print(result)
print('\n\n\n')
